## imports

In [99]:
from qiskit import QuantumCircuit , transpile
from qiskit_aer import AerSimulator
import random

In [100]:
#key_length = 20
def func(eve_intercepted,key_length):
    simulator=AerSimulator()
    alice_bits = [ random . randint (0 , 1) for _ in range ( key_length ) ]
    alice_bases = [ random . choice ([ 'X', 'Z']) for _ in range ( key_length ) ]
    bob_bases = [ random . choice ([ 'X','Z']) for _ in range ( key_length ) ]

    alice_qubits = []
    for i in range ( key_length ) :
        qc = QuantumCircuit (1 , 1)
   # Prepare the qubit in |0 > or |1 > based on alice_bits
        if alice_bits [ i ] == 1:
             qc.x (0)

        # Apply basis (X for Hadamard , Z for no operation )
        if alice_bases [ i ] == 'X':
            qc.h (0)

        alice_qubits.append(qc)
    eve_results = []
    for i , qc in enumerate ( alice_qubits ) :
        if eve_intercepted [ i ]:
            if random.choice(['X','Z']) =='Z':
                qc.h(0) # Measure in X basis
            qc.measure (0 , 0)
            transpiled_qc = transpile ( qc , simulator )
            result = simulator . run ( transpiled_qc ) . result ()
            counts = result . get_counts ()
    if counts . get ('0', 0) == counts . get ('1', 0) :
        eve_results . append (random.choice([0,1]))
    else :
        eve_results . append ( int ( max ( counts , key = counts . get ) ) )

    bob_results = []
    for i in range ( key_length ) :
        # Create Bob ’s circuit with Alice ’s prepared qubit
        qc = alice_qubits [ i ]. copy ()
        if bob_bases [ i ] == 'X':
            qc . h (0)
        qc . measure (0 , 0)

        transpiled_qc = transpile ( qc , simulator )
        result = simulator . run ( transpiled_qc ) . result ()
        counts = result.get_counts()
        if counts . get ('0', 0) == counts . get ('1', 0) :
            bob_results . append ( random . choice ([0 , 1]) )
        else :
            bob_results . append ( int ( max ( counts , key = counts . get ) ) )
    shared_key = []
    eve_successful_intercepts = 0
    total_matching_bases = 0

    for i in range ( key_length) :
        if alice_bases [ i ] == bob_bases [ i ]: # Keep only if bases match
            total_matching_bases += 1
            if alice_bits [ i ] == bob_results [ i ]: # Successful key bit
                shared_key . append ( alice_bits [ i ])
                if eve_intercepted [ i ]:
                    eve_successful_intercepts += 1

    if total_matching_bases > 0:
        eve_success_rate = eve_successful_intercepts / total_matching_bases
    else :  
        eve_success_rate = 0 # Avoid division by zero if no matching bases
    # Display results
    print (" Alice ’s Bits : ", alice_bits )
    print (" Alice ’s Bases : ", alice_bases )
    print (" Eve Intercepts : ", eve_intercepted )
    print (" Bob ’s Bases : ", bob_bases )
    print (" Bob ’s Results : ", bob_results )
    print (" Shared Key : ", [ int ( bit ) for bit in shared_key])
    print ( f" Eve ’s Success Rate : { eve_success_rate * 100}%")


    sample_size = len(shared_key)//5 +1 
    print(sample_size)
    sample_indices = random.sample(range(len(shared_key)), sample_size)

    print(sample_indices)
    errors = 0
    for index in sample_indices:
        if shared_key[index] != bob_results[index]:
            errors += 1 
    error_rate = errors / sample_size

    if error_rate > 0.1:#threshold for security
        print("High error rate detected. Possible eavesdropping. Key discarded.")
    else:
        print("Key is likely secure. Proceeding with the shared key.")

### Function calls

##### Function call 0%

In [109]:
key_len1 = 10
func(eve_intercepted1,key_len1)

 Alice ’s Bits :  [1, 1, 0, 0, 0, 0, 0, 0, 1, 1]
 Alice ’s Bases :  ['X', 'X', 'X', 'Z', 'Z', 'Z', 'Z', 'Z', 'X', 'X']
 Eve Intercepts :  [True, False, False, False, True, False, False, False, False, False]
 Bob ’s Bases :  ['Z', 'Z', 'X', 'X', 'X', 'Z', 'X', 'Z', 'X', 'X']
 Bob ’s Results :  [0, 1, 0, 1, 1, 0, 0, 0, 1, 1]
 Shared Key :  [0, 0, 0, 1, 1]
 Eve ’s Success Rate : 0.0%
2
[4, 3]
Key is likely secure. Proceeding with the shared key.


### Function call 20%

In [108]:
key_len2 = 10
eve_intercepted1= [random.choice ([True,True,False,False,False,False,False,False,False,False]) for _ in range ( key_len1 )]
updated2= random . choice ([0 , 1, 1])
func(eve_intercepted1,key_len1)

 Alice ’s Bits :  [1, 1, 1, 0, 1, 1, 1, 0, 1, 0]
 Alice ’s Bases :  ['X', 'X', 'Z', 'X', 'Z', 'X', 'Z', 'Z', 'X', 'X']
 Eve Intercepts :  [True, False, False, False, True, False, False, False, False, False]
 Bob ’s Bases :  ['X', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X']
 Bob ’s Results :  [1, 1, 1, 1, 0, 0, 1, 0, 1, 0]
 Shared Key :  [1, 1, 1, 1, 0, 0]
 Eve ’s Success Rate : 16.666666666666664%
2
[2, 5]
Key is likely secure. Proceeding with the shared key.


##### Function call 50%

In [102]:
key_len2 = 10
eve_intercepted1= [random.choice ([False,True]) for _ in range ( key_len1 )]
updated2= random . choice ([0 , 1])
func(eve_intercepted1,key_len1)

 Alice ’s Bits :  [0, 1, 0, 0, 1, 1, 1, 0, 1, 0]
 Alice ’s Bases :  ['X', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'Z']
 Eve Intercepts :  [True, False, True, False, True, False, False, False, False, True]
 Bob ’s Bases :  ['X', 'Z', 'Z', 'X', 'Z', 'Z', 'X', 'Z', 'Z', 'X']
 Bob ’s Results :  [0, 0, 1, 0, 0, 1, 1, 0, 1, 0]
 Shared Key :  [0, 0, 1, 1, 0, 1]
 Eve ’s Success Rate : 14.285714285714285%
2
[4, 5]
Key is likely secure. Proceeding with the shared key.


##### Function 80% 

In [ ]:
key_len2 = 10
eve_intercepted1= [random.choice ([False,True,True,True,True]) for _ in range ( key_len1 )]
updated2= random . choice ([0 , 1])
func(eve_intercepted1,key_len1)

 Alice ’s Bits :  [0, 1, 0, 0, 0, 1, 1, 0, 1, 0]
 Alice ’s Bases :  ['X', 'X', 'X', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z']
 Eve Intercepts :  [True, False, False, True, True, True, True, True, True, True]
 Bob ’s Bases :  ['Z', 'X', 'Z', 'X', 'Z', 'Z', 'Z', 'Z', 'X', 'Z']
 Bob ’s Results :  [0, 1, 1, 1, 0, 0, 1, 0, 1, 0]
 Shared Key :  [1, 0, 1, 0]
 Eve ’s Success Rate : 75.0%
1
[1]
High error rate detected. Possible eavesdropping. Key discarded.


##### 100%

In [ ]:
key_len2 = 10
eve_intercepted1= [random.choice ([True]) for _ in range ( key_len1 )]
updated2= random . choice ([0 , 1])
func(eve_intercepted1,key_len1)

 Alice ’s Bits :  [1, 0, 0, 0, 1, 1, 0, 0, 0, 0]
 Alice ’s Bases :  ['Z', 'X', 'Z', 'X', 'X', 'Z', 'X', 'Z', 'Z', 'X']
 Eve Intercepts :  [True, True, True, True, True, True, True, True, True, True]
 Bob ’s Bases :  ['Z', 'Z', 'Z', 'Z', 'X', 'X', 'X', 'X', 'Z', 'X']
 Bob ’s Results :  [1, 0, 0, 1, 1, 0, 0, 0, 0, 1]
 Shared Key :  [1, 0, 1, 0, 0]
 Eve ’s Success Rate : 83.33333333333334%
2
[2, 1]
High error rate detected. Possible eavesdropping. Key discarded.
